In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [ ]:
# RFSoC experiment configuration
# +----------------------------------------------------------------------------------------+
# | Feature: Single Channel 1.25 GHz BW Tx/Rx Experiment                                   |
# +-----------------+--------------------+----------------+-----------+-------+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO   | Ref. Clock |
# | 2.5 GSPS        | 2x                 | 1.25 GSPS      | 1.25G     | 625M  | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-------+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO   | Ref. Clock |
# | 2.5 GSPS        | 2x                 | 1.25 GSPS      | 1.25G     | -625M | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-------+------------+

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v39.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 2500.0,
    "DACInterpolationRate": 2,
    "DACNCO": 625.0,
    "ADCSampleRate": 2500.0,
    "ADCInterpolationRate": 2,
    "ADCNCO": -625.0,
    "CONFIG_NAME": "rfsoc_rfdc_v39"
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
rfdc_t = RfdcTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChTxTask(ol)
rx_t = SingleChRxTask(ol)

for task in [rfdc_t]:
    task.start()
    task.join()

parallel_task = [led_t, tx_t, rx_t]
for task in parallel_task:
    task.start()

time.sleep(5)

for task in parallel_task:
    task.stop()